# Find common syntax/word structures
We're going to look for common syntax structures among Linear A tablets. We classify each word/logogram
(including word separators) and come up with a list of tablets with common patterns.

First we load a lookup table of word types for classifying each word. We also have some helper functions

In [11]:
import csv                                                                                      
import json
import os
import re
from collections import Counter

def isNumber(word):
    if word >= u'\U00010100' and word <= u'\U0001013f':
        return True
    return False

place_names = {}
transaction_words = {}
transaction_signs = {}
numbers = {}
commodities = {}
from_suffix = {}
adjectives = {}
to_prefix = {}
fractions = {}
weights = {}
logograms = {}
words_in_linearb = {}
word_types = [(place_names, "060-place-names.txt", "place name", 0, 1),
        (transaction_words, "060-transaction-words.txt", "transaction term", 0, 1),
        (numbers, "050-numbers.csv", "number", 0, 1),
        (fractions, "065-fractions.txt", "fraction", 0, 0),
        (numbers, "065-fractions.txt", "number", 0, 0),
        (weights, "065-weights.txt", "weight", 0, 0),
        (logograms, "065-logograms.txt", "logogram", 0, 0),
        (commodities, "060-commodities.txt", "commodity", 0, 1),
        (from_suffix, "060-from-suffix.txt", "uses from suffix (-TE/-TI)", 0, 1),
        (adjectives, "060-adjectives.txt", "adjective for placename", 0, 1),
        (to_prefix, "060-to-prefix.txt", "uses to prefix (I-/J-)", 0, 1),
        (words_in_linearb, "135-identical-words-in-linearb.txt", "word also in linear b", 0, 1),
        (transaction_signs, "160-transaction-signs.txt", "transaction sign", 0, 1),
        ]
for word_type in word_types:
    dictionary = word_type[0]
    input_file = open("AnalyseWordPatternsData/" + word_type[1], 'r')
    while True:
        line = input_file.readline()
        if not line:
            break
        line_array = line.strip().split('\t')
        dictionary[line_array[word_type[3]]] = line_array[word_type[4]]

json_file = open('AnalyseWordPatternsData/150-metadata-template.js')
data = json.load(json_file)
inscriptions = data["metadata"]

def isHeadWord(word, original_word, no_of_words):
    if no_of_words == 1:
        return False
    if index:
        return False
    if original_word.startswith(u'\U0001076b'):
        return False
    if original_word == u'\U00010101':
        return False
    if word == "—":
        return False
    if word.isnumeric():
        return False
    return True;

def assignNumberToPreviousWord(word, word_tags, index, prev_word_tag, prev_original_word):
    if word not in numbers:
        return False
    if not index:
        return False
    if word == u'\U00010101':
        return False
    if word.startswith(u'\U0001076b'):
        return False
    cleaned_prev_original_word = prev_original_word.replace(u'\U0001076b', "")
    if cleaned_prev_original_word in numbers:
        return False
    if prev_original_word.endswith(u'\U0001076b'):
        return False
    if prev_original_word == "\n":
        return False
    return True

def shouldIncludeWord(word):
    if word in numbers:
        return False
    if u'\U00010101' in word:
        return False
    if word == "—":
        return False
    if word == "":
        return False
    return True

def wordRepeatedInInscription(word_tags, word):
    if not shouldIncludeWord(word):
        return False

    words = list(map(lambda x: x["word"].replace(u'\U0001076b', ""), word_tags))
    if words.count(word) > 1:
        return True
    return False

word_find_spots = {}
def addFindSpot(name, word):
    if not shouldIncludeWord(word):
        return
    find_spot = name[:2]
    if word not in word_find_spots:
        word_find_spots[word] = [find_spot]
        return
    word_find_spots[word].append(find_spot)

for inscription in inscriptions:
    for tags in inscription["tagsForWords"]:
        word = tags["word"]
        word = word.replace(u'\U0001076b', "")
        addFindSpot(inscription["name"], word)

locations = {}
locations["ZA"] ="Zakros"          
locations["PK"] ="Palaikastro"  
locations["PE"] ="Petras"  
locations["SY"] ="Syme" 
locations["PS"] ="Pseira" 
locations["MA" ] ="Malia" 
locations["AR"] ="Arkhalkhori" 
locations["IO"] ="Iouktas" 
locations["KN"] ="Knossos" 
locations["TY"] ="Tylissos"  
locations["PH"] ="Phaistos"  
locations["HT"] ="Haghia Triada"  
locations["AP"] ="Apodoulou"  
locations["KH"] ="Kharnia" 



## Create a list of all word structures

First create a list of word structures keyed by tablet name. Each word has a number of possible classifications so the word structure is a list of lists, for example the word structure for HT1 might look like:
```
'HT1': [['word'],
  ['word separator'],
  ['transaction term', 'transaction sign', 'word'],
  ['number'],
  ['word'],
  ['number'],
  ['word'],
  ['number'],
  ['word'],
  ['number'],
  ['word'],
  ['number']]
```

In [12]:
# Create a dictionary data structure containing word structure lists for all tablets.

word_structures = {}
new_inscriptions = []
for old_inscription in inscriptions:
    inscription = old_inscription.copy()
    word_tags = inscription["tagsForWords"]
    
    inscription["transactions"] = {}
    current_pattern = []
    
    for index, word_tag in enumerate(word_tags):
        word = word_tag["transliteratedWord"]
        original_word = word_tag["word"]
        if "tags" in word_tag:
            del word_tag["tags"]

        tags = []
        if word == "\n":
            continue

        if u'\U00010101' in original_word:
            tags.append("word separator")
        if original_word == "—":
            tags.append("dividing line")

        cleaned_word = word.replace(u'\U0001076b', "")
        cleaned_original_word = original_word.replace(u'\U0001076b', "")
        for word_type in word_types:
            dictionary = word_type[0]
            annotation = word_type[2]
            if cleaned_word in dictionary or cleaned_original_word in dictionary:
                if not annotation in tags:
                    tags.append(annotation)

        if (len(cleaned_word) > 1 and not "word separator" in tags
            and word != '—' and word !=  u'\U0001076b'
            and not 'logogram' in tags
            and not 'commodity' in tags
            and not "number" in tags and not "fraction" in tags):
            tags.append("word")

        if original_word == u'\U0001076b':
            tags.append("lacuna")
        else:
            for i, tag in enumerate(tags):
                if original_word.startswith(u'\U0001076b'):
                    tags[i] = "]" + tags[i]
                if original_word.endswith(u'\U0001076b'):
                    tags[i] = tags[i] + "["

        current_pattern += [sorted(tags)]
  
    inscription["words"] = inscription.pop("tagsForWords")
    word_structures[inscription["name"]] = current_pattern
    


Now that we have an exhaustive list of word structures, we strip out some of the word-tags so that we don't have too many permutations. We create `stripped_word_structures` for this.

In [13]:
import itertools

# To reduce the number of permutations we strip out word descriptions that are too specific.
def containsRedundantWord(l,
    words_to_drop = ["lacuna at", "transaction", "fraction", "weight", "commodity"]
    ):
    for w in words_to_drop:
        if w in l:
            return True
    return False

stripped_word_structures = {k: [[w for w in wl if not containsRedundantWord(w)]
                                for wl in v] 
                            for k,v in word_structures.items()}

def hasLacuna(l):
    return containsRedundantWord(l, ["lacuna"])

Create a list of word-tag permutations for each tablet. This will give us a list of different word-sequences for the tablet. We will use this to look for common sequences among all the tablets.

In [8]:
#small_sws = dict(list(stripped_word_structures.items())[0: 840])
#[list(x) for x in itertools.product(*stripped_word_structures['HT2'])]

# Create permutations for each tablet. It's a list of lists for each tablet.
ws_perms = {k: [list(x) for x in itertools.product(*v)]
             for k,v in stripped_word_structures.items()}
#dict(list(ws_perms.items())[0:2])

# Check for entries with too many permutations.
for k,v in ws_perms.items():
    if len(v) < 10:
        continue
    print(k, len(v))


Get all common sequences among the tablets and store it in `matched_sequences`.

In [9]:
%%time
from difflib import SequenceMatcher
from collections import defaultdict
from IPython.display import display, clear_output

"""To compare each tablet with every other tablet we take a copy of ws_perms and
use that as the one to check each tablet against. We delete each tablet from
the copy as we use it so that we're not comparing tablets more than once."""
ws_perms_copy = ws_perms.copy()
matched_sequences = defaultdict(list)
for k,v in ws_perms.items():
    # Delete from the copy so we don't perform unnecessary comparisons.
    del ws_perms_copy[k]
    # Check each permutation..
    for l in v:
        clear_output(wait=True)
        display(str(len(ws_perms) - len(ws_perms_copy)) + ' done.')
        s = SequenceMatcher(None)
        s.set_seq1(l)
        # .. against each permutation in the rest of the list.
        for k1,v1 in ws_perms_copy.items():
            for l1 in v1:
                s.set_seq2(l1)
                r = s.get_matching_blocks()
                for m in r:
                    # Ignore any sequences shorter than 3.
                    if m.size < 3:
                        continue
                    structure = ','.join(l[m.a:m.a+m.size])
                    matched_sequences[structure] += [(k,m.a,m.size), (k1,m.b,m.size)]
                    matched_sequences[structure] = list(set(matched_sequences[structure]))


'1712 done.'

CPU times: user 33 s, sys: 1.51 s, total: 34.5 s
Wall time: 37.3 s


Now we have enough data to print out matching sequences. We're going to start by looking at sequences at the start of inscriptions.

In [17]:
def getSyntacticalStructures(matched_sequences
                             , from_start_only=False
                             , min_length=2
                             , min_results=2):
    """Get all instances of common syntatical structure, either `from_start_only` or not, with at least"
    `min_results` matches of `min_length`"""
    # Create a list of words in each tablet that we can look up by tablet name.
    la_words = {inscription["name"] : [v["word"] for v in inscription["tagsForWords"]
                                       if v["word"] != '\n'] 
                for inscription in inscriptions}
    en_words = {inscription["name"] : [v["transliteratedWord"] for v in inscription["tagsForWords"]
                                       if v["word"] != '\n'] 
                for inscription in inscriptions}

    structures = {}
    # We're looking for sequences of 3 words or more at the start of the inscription.
    for i in range(50,min_length,-1):
        test_seq = {k:v for k,v in matched_sequences.items() if len(k.split(',')) == i}
        for k,v in test_seq.items():
            en_seqs = {}
            for tablet, start, end in v:
                # We're only looking at structures at the start of the inscription.
                if start and from_start_only:
                    continue
                en_seqs[tablet] = en_words[tablet][start:start+end]

            # Ignore any cases where we don't have `min_results` inscriptions with a matching sequence.
            if len(en_seqs.keys()) < min_results:
                continue

            structures[k] = en_seqs
    return structures



Create tables of all word sequences at the start of inscriptions.

In [18]:
import pandas as pd
styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])]

syn_seqs = getSyntacticalStructures(matched_sequences, True)
for k, en_seqs in syn_seqs.items():
    df = pd.DataFrame(en_seqs.values(), columns = k.split(','))
    df.set_axis(en_seqs.keys(), axis='index', inplace=True)
    #df.style.set_caption("Davis 2018 Mapping of Linear A and PD Symbols").set_table_styles(styles)
    display(df)

,word,word separator,word,word separator,logogram,number,logogram,number,logogram,number,...,number,logogram,number,word,logogram,number,logogram,number,logogram,number
HT14,PU-VIN,𐄁,TE,𐄁,GRA,30,OLE+MI,3,OLE+DI,3,...,9,OLIV,13,A-PU₂-NA-DU,GRA,45,OLE+MI,5,OLE+DI,4
HT28a,A-SI-JA-KA,𐄁,JA-*21F,𐄁,GRA+QE,5,OLE+U,2,OLE+KI,¹⁄₂,...,1,OLE+TU,𐝉,SA-RA₂,OLE+DI,1,NI,2,VIN,3


,word,word separator,logogram,word,number,word,number,word,number,word,number,word,number
HT7a,QE-TI,𐄁,VIR+[?],I-RU-JA,3,DU-JA,4,TA-NA-TI,1,DA-RE,1,TE-TU,1
HT95a,DA-DU-MA-TA,𐄁,GRA,DA-ME,10,MI-NU-TE,10,SA-RU,20,KU-NI-SU,10,DI-DE-RU,10


,word,word separator,word,number,word,number,word,number,word,number,word,number
HT94b,KI-RO,𐄁,TU-MA,1,PA-TA-NE,1,DE-DI,1,KE-KI-RU,1,SA-RU,1
HT1,QE-RA₂-U,𐄁,KI-RO,197,ZU-SU,70,DI-DI-ZA-KE,52,KU-PA₃-NU,109,A-RA-NA-RE,105


,word,word separator,word,word separator,logogram,number,logogram,number,logogram,number
HT14,PU-VIN,𐄁,TE,𐄁,GRA,30,OLE+MI,3,OLE+DI,3
HT21,PI-TA-KA-SE,𐄁,TE,𐄁,GRA,161,OLE+U,11,OLE+E,3
HT28a,A-SI-JA-KA,𐄁,JA-*21F,𐄁,GRA+QE,5,OLE+U,2,OLE+KI,¹⁄₂


,word,logogram,number,number,logogram,number,logogram,number,word,logogram
HT129,KI-RE-TA₂,GRA,33,𐝆𐝁,*304,10,NI,22,TU-*21F-RI-NA,GRA
ZA6b,I-KU-JU-TI-*28B,GRA+PA,50,¹⁄₄,VIN+RA,40,VIN+KA,10,*312-TA₂,GRA+PA


...

,lacuna,number,number
GO2v,𐝫,1,¹⁄₈
PE2,𐝫,3,¹⁄₃


,]word,logogram,word separator
PKZa14,TU-ME,I,𐄁
PH1a,DI-RA-DI-NA,*316,𐄁


,word,word,word
SAMWa1,JA-SA,SA-RA,TE
PRZa1,TA-NA-SU-TE-KE,SE-TO-I-JA,A-SA-SA-*802-ME


,lacuna,]word[,word separator
PKZa9,𐝫,JA-U-PA-MA-I-DA,𐄁
PH28a,𐝫,A-PA₃,𐄁


,]word,word,word separator
PSZa2,RE-I-KE,TA-NA-I-*301-TI,𐄁
VRYZa1,I-PI-NA-MA,SI-RU-TE,𐄁


Create a list of all sequences.

In [19]:
syn_seqs = getSyntacticalStructures(matched_sequences,False)
for k, en_seqs in syn_seqs.items():
    df = pd.DataFrame(en_seqs.values(), columns = k.split(','))
    df.set_axis(en_seqs.keys(), axis='index', inplace=True)
    #df.style.set_caption("Davis 2018 Mapping of Linear A and PD Symbols").set_table_styles(styles)
    display(df)

,word,word separator,word,word separator,logogram,number,logogram,number,logogram,number,...,number,logogram,number,word,logogram,number,logogram,number,logogram,number
HT14,PU-VIN,𐄁,TE,𐄁,GRA,30,OLE+MI,3,OLE+DI,3,...,9,OLIV,13,A-PU₂-NA-DU,GRA,45,OLE+MI,5,OLE+DI,4
HT28a,A-SI-JA-KA,𐄁,JA-*21F,𐄁,GRA+QE,5,OLE+U,2,OLE+KI,¹⁄₂,...,1,OLE+TU,𐝉,SA-RA₂,OLE+DI,1,NI,2,VIN,3


,number,word,number,word,number,word,number,word,number,word,number,word,number,word,number,word,number,word,number
HT117a,1,MI-TU,1,KU-RA-MU,1,MA-RU,1,KU-PA₃-NU,1,TU-JU-MA,1,U-DI-MI,1,MI-RU-TA-RA-RE,1,TE-JA-RE,1,NA-DA-RE,1
HT122a,1,TE-KI,2,QA-*310-I,3,JA-MI-DA-RE,1,SI-DA-RE,1,*324-DI-RA,1,PA-DE,1,KU-PA₃-NU,1,PA-TA-NE,1,*306-TU,1


,word,word separator,word,number,word,number,word,number,word,number,word,number,word,number,word,number,word,number
HT9b,KA-*305,𐄁,PA-DE,3,A-SI,3,*306-TU,8,*324-DI-RA,2,QE-PU,2,TA-I-AROM,2,DI-NA-U,4,KU-RO,24
HT117a,U-MI-NA-SI,𐄁,U-SU,1,MI-TU,1,KU-RA-MU,1,MA-RU,1,KU-PA₃-NU,1,TU-JU-MA,1,U-DI-MI,1,MI-RU-TA-RA-RE,1


,word,word separator,word also in linear b,number,word,number,word,number,word,number,word,number,word,number,word,number,word,number
HT9b,KA-*305,𐄁,PA-DE,3,A-SI,3,*306-TU,8,*324-DI-RA,2,QE-PU,2,TA-I-AROM,2,DI-NA-U,4,KU-RO,24
HT117a,U-MI-NA-SI,𐄁,U-SU,1,MI-TU,1,KU-RA-MU,1,MA-RU,1,KU-PA₃-NU,1,TU-JU-MA,1,U-DI-MI,1,MI-RU-TA-RA-RE,1


,logogram,number,logogram,number,logogram,number,logogram,number,logogram,number,word,logogram,number,logogram,number,logogram,number,word
HT28a,GRA+QE,5,OLE+U,2,OLE+KI,¹⁄₂,OLE+MI,1,OLE+TU,𐝉,SA-RA₂,OLE+DI,1,NI,2,VIN,3,VIR+KA-VIN
KH11,CYP+E,𐝇𐝉,VIN,≈ ¹⁄₆,*306,4,CYP,¹⁄₃,*348-CYP,𐝇𐝉,A-TO-*349-TO-I,CYP+E,3,NI,1,VIN,3,A-TA-*350


...

,word,number[,number
ZA8,A-RI-NI-TA,1,?
PH12a,VIR+[?]-*339-HIDE+[?],10,𐝈𐝈


,lacuna,]word[,word separator
PKZa9,𐝫,JA-U-PA-MA-I-DA,𐄁
PH28a,𐝫,A-PA₃,𐄁


,word separator,word,word[
PKZa8,𐄁,TU-ME-I,JA-SA-U-NA-KA-NA-SI
PKZa16,𐄁,PU₂,RE-JA


,]word,word,word separator
PSZa2,RE-I-KE,TA-NA-I-*301-TI,𐄁
VRYZa1,I-PI-NA-MA,SI-RU-TE,𐄁


,word[,number[,logogram
ZA12b,PA,40,TI
ZA12a,PA,64,TI


Create a table of sequences in descending order of occurence.

In [82]:
mat_seq_cnt = {k: len(v) for k,v in matched_sequences.items()}

pd.set_option("display.max_rows", None, "display.max_columns", None)
df = pd.DataFrame(mat_seq_cnt.values(), columns = ["Count"])
df.set_axis(mat_seq_cnt.keys(), axis='index', inplace=True)
#df.style.set_caption("Davis 2018 Mapping of Linear A and PD Symbols").set_table_styles(styles)
display(df.sort_values(by=['Count'], ascending=False))



,Count
"word,number,word,number",138
"logogram,number,logogram,number",119
"logogram,number,logogram",115
"word,number,word",106
"word,word separator,word",78
"logogram,number,word",73
"word,word separator,logogram",52
"word,logogram,number",48
"logogram,number,number",46
"word,word separator,word,word separator",43


In [81]:
import re

# Drop sequences that start with a number or a word separator
matched_sequences = {k:v for k,v in matched_sequences.items()
                     if not k.startswith("number") and not k.startswith("word sep")}

# Replace sequences of 'word/logogram,number' with 'word/logogram number sequence'.
regex_matched_sequences = defaultdict(list)
for k,v in matched_sequences.items():
    nk = re.sub('((logogram|word),number(,)?)+', 'word/logogram number sequence,', k)
    regex_matched_sequences[nk[:-1] if nk.endswith(',') else nk] += [k]

# Get and print a count of these.
mat_seq_cnt = {k: sum([len(matched_sequences[seq]) for seq in seqs])
               for k,seqs in regex_matched_sequences.items()}

df = pd.DataFrame(mat_seq_cnt.values(), columns = ["Count"])
df.set_axis(mat_seq_cnt.keys(), axis='index', inplace=True)
display(df.sort_values(by=['Count'], ascending=False))


,Count
word/logogram number sequence,562
"word/logogram number sequence,word",322
"word/logogram number sequence,logogram",164
"word,word separator,word/logogram number sequence",108
"word,word/logogram number sequence",101
"word/logogram number sequence,number",95
"word,word separator,word",78
"word/logogram number sequence,number,word/logogram number sequence",72
"word/logogram number sequence,word,word/logogram number sequence",57
"word,word separator,logogram",52


In [79]:
string = "jklfds,"
string[:-1]

'jklfds'

In [73]:
import re
l = "term,logogram,number,word,number,word,number,logogram,number,word,number"
re.sub('((logogram|word),number(,)?)+', r'sequence\3',l)


'term,sequence,'

In [229]:
word_structures["HT24a"]

[['word'],
 ['logogram['],
 [']number'],
 ['logogram'],
 ['number'],
 [']number'],
 ['fraction', 'number'],
 ['logogram'],
 ['number'],
 ['adjective for placename', 'word'],
 ['logogram['],
 ['word'],
 ['logogram[']]

In [248]:
{20} & {20, 10} 

{20}

In [18]:
# Write out a template file for the selected inscription
name = "HT86a"
output_file = open(name + ".js", "w")
output_file.write(json.dumps([x for x in inscriptions if x["name"] == name], sort_keys=True, indent=4, ensure_ascii=False))
output_file.close()


